In [1]:
def data_extract(year,month_list):
    # year: year in string format
    # month_list: list of months in string format
    data_appended = pd.DataFrame() #collects all data of desired year and months
    for month in month_list:
        x = pd.read_csv(data_path+"//"+year+month+".csv")
        data_appended = data_appended.append(x)
    return data_appended
    
def string2datetime(time_col):
    time_list = [str.split(list(time_col)[i])[0] for i in range(len(time_col))]
    return pd.to_datetime(time_list)

In [3]:
#feature distribution and correlation

def import_station_data(path,station_name):
    station_data = pd.read_csv(path+"/"+station_name+".csv").drop("Unnamed: 0",axis=1)
    return station_data

def time_coversion(time_col): #Converts time from string to datetime
    time_list = [time_col[i].split()[0] for i in range(len(time_col))]
    return pd.to_datetime(time_list)

def trend_plot(cols,years,data):
    plt.figure(figsize=(30,15))
    for i in range(len(years)):
        temp = data[data['time'].dt.year==years[i]]
        plt.subplot(2,3,i+1)
        for col in cols:
            plt.plot(temp.time,temp[col],'o',label=col)
        plt.legend()
        plt.xticks(rotation=90)
    name = ''
    for col in cols:
        name = name + col+' '
        
#     plt.savefig(result_path+'/Year wise trend of '+name+'.png')

def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    ax.annotate(f'ρ = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)



def scatter_plot(result_path,data,label):
    g = sns.pairplot(data,corner=True)
    g.fig.suptitle(label)
    g.map_lower(corrfunc)
    
    plt.savefig(result_path+"//Pairwise scatter plot for "+label+".png")


def generate_pairplot(MLdata_path,result_path,station_list,label,col_investigate):
    for station in station_list:
        print(station)
        data = import_station_data(MLdata_path,station)
        data.time = time_coversion(data.time)

        #### Temprature
        data_investigate = data[col_investigate].dropna()
        if len(data_investigate)==0:
            continue
        
        folder_path = result_path+"/"+label
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Pairwise scatter plot 
        scatter_plot(folder_path,data_investigate,station)

        plt.close()


def make_gif(result_path,label):
    import glob
    from PIL import Image
    frame_folder = result_path+"/"+label
    frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.png")]
    frame_one = frames[0]
    frame_one.save(frame_folder+"/"+label+".gif", format="GIF", append_images=frames,
               save_all=True, duration=100, loop=0)